In [2]:
!rm -rf ~/.cache/huggingface/datasets/ag_news
# 🚀 STEP 1: Install dependencies
!pip install transformers peft datasets accelerate bitsandbytes

# 🚀 STEP 2: Import libraries
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset

# 🚀 STEP 3: Load model + tokenizer
model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)

# 🚀 STEP 4: Prepare model for LoRA fine-tuning
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# 🚀 STEP 5: Load & tokenize SQuAD dataset (small slice for demo)
dataset = load_dataset("ag_news", split="train[:0.5%]")

def tokenize_function(example):
    return tokenizer(
        f"Question: {example['question']} Context: {example['context']} Answer: {example['answers']['text'][0]}",
        truncation=True
    )
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 🚀 STEP 6: Set up training
training_args = TrainingArguments(
    output_dir="./asked_model",
    per_device_train_batch_size=2,
    learning_rate=2e-4,
    num_train_epochs=1,
    save_steps=10,
    save_total_limit=2
)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

# 🚀 STEP 7: Fine-tune the model
trainer.train()

# 🚀 STEP 8: Save the fine-tuned model
model.save_pretrained("./asked_model")
tokenizer.save_pretrained("./asked_model")

# 🚀 STEP 9: Load and chat with the fine-tuned model
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("./asked_model")
model = AutoModelForCausalLM.from_pretrained("./asked_model", device_map="auto")

while True:
    question = input("Ask a question (or type 'exit' to quit): ")
    if question.lower() == 'exit':
        break
    inputs = tokenizer(question, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=50)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"\nAnswer: {answer}\n")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [10]:
# Remove Hugging Face dataset cache — this clears corrupted or partial downloads
!rm -rf ~/.cache/huggingface/datasets
!rm -rf ./hf_cache
!rm -rf squad
!rm -rf ag_news


In [12]:
from datasets import load_dataset

# Download fresh dataset, forcing a redownload to avoid cache issues
dataset = load_dataset("ag_news", split="train[:0.5%]", download_mode="force_redownload")

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [13]:
from datasets import load_dataset
import tempfile
import os

# Create a temporary directory for the dataset cache
with tempfile.TemporaryDirectory() as tmpdir:
    print(f"Using temporary cache directory: {tmpdir}")
    # Download fresh dataset, forcing a redownload to avoid cache issues
    # Use the temporary directory as the cache directory
    dataset = load_dataset("ag_news", split="train[:0.5%]", download_mode="force_redownload", cache_dir=tmpdir)

def tokenize_function(example):
    # Make sure the tokenizer variable is still available in this scope if needed
    return tokenizer(example["text"], truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Using temporary cache directory: /tmp/tmp6rmyktkd


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [16]:
from datasets import load_dataset

# Download fresh dataset, forcing a redownload to avoid cache issues
# Remove the cache_dir parameter to use the default Hugging Face cache location
dataset = load_dataset("ag_news", split="train[:0.5%]", download_mode="force_redownload")

def tokenize_function(example):
    # Make sure the tokenizer variable is still available in this scope if needed
    return tokenizer(example["text"], truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [17]:
from datasets import load_dataset
import tempfile

with tempfile.TemporaryDirectory() as tmpdir:
    print(f"Using temp cache: {tmpdir}")
    dataset = load_dataset("ag_news", split="train[:0.5%]", cache_dir=tmpdir, download_mode="force_redownload")


Using temp cache: /tmp/tmpdswpkrfl


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [15]:
!rm -rf ~/.cache/huggingface/datasets/ag_news
